## [Url](https://imbalanced-learn.readthedocs.io/en/stable/combine.html)

## SMOTE

* marginal outlier 와 inliner 사이에 새로운 데이터 포인트를 생성시킨다. 
* 이것은 oversampling 으로 부터 발생하는 공간을 지울 수 있으므로 해결할 수 있다?
* The two ready-to use classes imbalanced-learn implements for combining over- and undersampling methods are: (i) `SMOTETomek` [BPM2004] and (ii) `SMOTEENN`

In [3]:
from collections import Counter
from sklearn.datasets import make_classification

In [5]:
X, y = make_classification(n_samples=5000, n_features=2, n_informative=2,
                            n_redundant=0, n_repeated=0, n_classes=3,
                            n_clusters_per_class=1,
                            weights=[0.01, 0.05, 0.94],
                            class_sep=0.8, random_state=0)

In [7]:
print( sorted( Counter(y).items()))

[(0, 64), (1, 262), (2, 4674)]


In [8]:
from imblearn.combine import SMOTEENN

In [10]:
smote_enn = SMOTEENN( random_state= 0 )
X_resampled , y_resampled = smote_enn.fit_resample(X , y)

print(sorted(Counter(y_resampled).items()))

[(0, 4060), (1, 4381), (2, 3502)]


## Ensemble of samplers 

### 5.1 Classifier including inner balancing samplers 

#### 5.1.1 Bagging Classifier

* `BaggingClassifer` 가 있는데, 그러나 이것은 데이터의 균형을 맞춰주지 못하고 대다수 class에 더 선호한다.

`balanced_accuract_score`

* `BalancedBaggingClassifier` 은 각각의 subset에서 데이터를 학습전에 Resample 한다. 

* `Bagging` 이란 원래 데이터를 여러개의 작은 N개를 샘플링해서 만든 다음 각각의 작은 데이터로 모델링을 하고 이것을 합쳐서 최종적인 모델로 사용한다는 개념

* `Boosting` 은 간단한 모델로 전체를 학습시킨 다음, 간단한 모델이다 보니 에러가 남을텐데, 거기 중에서 에러가 큰 부분에 대해서 다시 모아서 다시 새로운 간단한 모델을 학습시키는 것을 반복한다. 


In [20]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
bc = BaggingClassifier(base_estimator = DecisionTreeClassifier(), random_state = 0)

bc.fit(X_train , y_train)

y_pred = bc.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.8864390007810891

In [21]:
from imblearn.ensemble import BalancedBaggingClassifier
bbc = BalancedBaggingClassifier(base_estimator=DecisionTreeClassifier(),
                             sampling_strategy='auto',
                             replacement=False,
                             random_state=0)
bbc.fit(X_train , y_train)

y_pred = bbc.predict(X_test)
balanced_accuracy_score(y_test , y_pred)

0.8691879549364346

## 5.1.2  Forest of Randomized trees 

* `BalancedRandomForestClassifier` 은 각각의 트리에서 Balanced bootstrap sample 해서 생성되는 또다른 앙상블 방법이다.


In [22]:
from imblearn.ensemble import BalancedRandomForestClassifier
brf = BalancedRandomForestClassifier(n_estimators=100, random_state=0)
brf.fit(X_train, y_train) 
y_pred = brf.predict(X_test)
balanced_accuracy_score(y_test, y_pred)  



0.8779524446169549

In [24]:
brf.feature_importances_ 

array([0.5976795, 0.4023205])

## 5.1.3 Boosting 

`RUSBoostClassifier` 은 boosting iteration을 하기 전에 데이터를 랜덤하게 undersample 한다.

In [25]:
from imblearn.ensemble import RUSBoostClassifier
rusboost = RUSBoostClassifier(random_state=0)
rusboost.fit(X_train, y_train)  
y_pred = rusboost.predict(X_test)
balanced_accuracy_score(y_test, y_pred)  

0.8134910366440966

`AdaBoost` 은 bagging classifier 로 사용되고 `EasyEnsemble` 로 불린다.
`EasyEnsembleClassifie` 는 adaboost 방법을 사용한다. 
*  BalancedBaggingClassifier 와 유사하다.

In [26]:
from imblearn.ensemble import EasyEnsembleClassifier
eec = EasyEnsembleClassifier(random_state=0)
eec.fit(X_train, y_train) 
y_pred = eec.predict(X_test)
balanced_accuracy_score(y_test, y_pred)  

0.7931491784189416